In [1]:
#load libraries
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import datetime as datetime
import matplotlib.colors as mcolors
import seaborn as sns

In [2]:
#load ML specific libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import interpolate

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
hemi = 'NH' #options: 'NH', 'SH'
id = 1 
feature = 'fac' #options: 'pot', 'fac', 'sxx', 'syy', 'sxy'

In [5]:
#ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/test/half_day_1deg.nc')
ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/multi/2020-2024_{feature}_feats_140_40_2min_rtsw.nc')
#ds = ds.sel(dt=slice('2023-01-01', '2024-12-31'))
ds

<xarray.Dataset> Size: 23GB
Dimensions:     (dt: 1024732, lat: 40, lon: 140)
Coordinates:
  * dt          (dt) datetime64[ns] 8MB 2020-08-01 ... 2024-07-31T23:58:00
  * lon         (lon) float64 1kB 0.0 2.59 5.18 7.77 ... 352.2 354.8 357.4 360.0
  * lat         (lat) float64 320B 49.5 51.14 52.78 54.42 ... 88.34 89.17 90.0
Data variables:
    fac         (dt, lat, lon) float32 23GB ...
    np          (dt) float64 8MB ...
    vsw         (dt) float64 8MB ...
    By          (dt) float64 8MB ...
    Bz          (dt) float64 8MB ...
    tilt_angle  (dt) float64 8MB ...

In [7]:
#check for NaNs in the dataset
#good sanity check, but not necessary
def nan_count(ds, var):
    #count number of NaNs in a data variable
    con_data = ds[var].values
    nan_count = np.count_nonzero(np.isnan(con_data))
    nan_ratio = nan_count / con_data.size
    not_nan = con_data.size - nan_count
    return not_nan, nan_count, np.round(nan_ratio, 2)

#nan_count(ds, 'np')

In [8]:
# Initialize scalers for the target variable and input variables
input_scaler = StandardScaler() #scale using standard dev where mean is 0 and std is 1

# Extract the target variable and reshape for scaling
target_var = ds[feature].values  # shape (t (n), mlat (50), mlt (24))

# Extract and scale input variables (variables that are dependent only on 'dt')
input_vars = ['By', 'Bz', 'vsw', 'np', 'tilt_angle'] #RTSW
#input_vars = ['BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', 'tilt_angle'] #OMNI

input_data = np.array([ds[var].values for var in input_vars]).T  # shape (22320, number_of_vars)
input_data_scaled = input_scaler.fit_transform(input_data)

file_path = f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_scaler_{feature}_{hemi}_id{id}.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(input_scaler, file)

def create_sequences(target_data, input_data, lb):
    X, y = [], []
    for i in range(len(target_data) - lb):
        X.append(input_data[i:i+lb].T)
        y.append(target_data[i+lb])

    return np.array(X), np.array(y)

lookback = 30 # number of time steps to look back. At 1 min set to 60, at 2 min set to 30, and so on
X, y = create_sequences(target_var, input_data_scaled, lookback)

X.shape, y.shape

((1024673, 6, 30), (1024673, 40, 140))

In [10]:
def simple_time_series_split(X, y):

    #split the data into training, validation, and test sets
    #this is based on analysis of the solar wind speed data across the period
    X_train_1, X_test, X_train_2, X_val, X_train_3 = X[:350000], X[350000:450000], X[450000:750000], X[750000:850000], X[850000:]
    y_train_1, y_test, y_train_2, y_val, y_train_3 = y[:350000], y[350000:450000], y[450000:750000], y[750000:850000], y[850000:]
    X_train = np.concatenate((X_train_1, X_train_2, X_train_3), axis=0)
    y_train = np.concatenate((y_train_1, y_train_2, y_train_3), axis=0)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = simple_time_series_split(X, y)
X_train.shape, X_val.shape, X_test.shape

((824673, 6, 30), (100000, 6, 30), (100000, 6, 30))

In [11]:
# Convert data to PyTorch tensors and move to GPU
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = np.array(y_test)

# Create DataLoader
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [12]:
mlat_len = np.size(ds['lat'].values)
mlt_len = np.size(ds['lon'].values)

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, mlat_len * mlt_len)

    def forward(self, x):
        batch_size = x.size(0)
        # Use the same device as the model for hidden states
        device = next(self.parameters()).device  # Ensure hidden states are on the same device as the model
        
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size, device=device)  # Initial hidden state
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size, device=device)  # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = out.view(-1, mlat_len, mlt_len)
        return out

    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

# Instantiate the model with lookback size
hs = 128   #hidden state
sl = 2     #stacked layers
model = LSTM(lookback, hs, sl).to(device)
model

LSTM(
  (lstm): LSTM(30, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=5600, bias=True)
)

In [14]:
#calculate the area for a given latitude and longitude
def calculate_delta(lat1, lon1, lat2, lon2):
    lat1 = np.abs(lat1)
    lat2 = np.abs(lat2)
    
    radius = 6371.008 * 1000

    area = radius**2 * (np.sin(np.radians(lat2)) - np.sin(np.radians(lat1))) * (np.radians(lon2) - np.radians(lon1))
    
    return area 

#set up the latitude and longitude intervals
mlat = ds['lat'].values
mlt = ds['lon'].values

#calculate the area for each latitude and longitude interval
def calculate_area(mlat, mlt):
    areas = np.zeros((mlat_len-1, mlt_len-1))
    for i in range(len(mlat) - 1):
        for j in range(len(mlt) - 1):
            lat1, lat2 = mlat[i], mlat[i + 1]
            lon1, lon2 = mlt[j], mlt[j + 1]

            area = calculate_delta(lat1, lon1, lat2, lon2)
            areas[i, j] = area

    #interpolate the areas back to shape of mlat, mlt
    #Currently 49x23, interpolate back to 50x24
    x = np.arange(areas.shape[1])
    y = np.arange(areas.shape[0])
    f = interpolate.interp2d(x, y, areas, kind='linear')
    xnew = np.arange(0, areas.shape[1], areas.shape[1]/(areas.shape[1]+1))
    ynew = np.arange(0, areas.shape[0], areas.shape[0]/(areas.shape[0]+1))
    areas = f(xnew, ynew)

    return areas

areas = calculate_area(mlat, mlt)

def weighted_loss_function(y_true, y_pred):

    # Normalize the area
    weights = areas / np.min(areas)

    #create custom 'Mean Area Weighted Loss' (MAWE) function
    loss = torch.mean(torch.abs(y_true - y_pred) * torch.tensor(weights, dtype=torch.float32).to(device))
    
    return loss

/tmp/ipykernel_20412/1567071016.py:31: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interpolate.interp2d(x, y, areas, kind='linear')
/tmp/ipykernel_20412/1567071016.py:34: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator

In [15]:
#ptimizer = torch.optim.Adam(model.parameters(), lr=7e-4) # Adam optimizer
#scheduler = StepLR(optimizer, step_size=8, gamma=0.7) #decay learning rate by 0.7 every 8 epochs
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # Adam optimizer
scheduler = StepLR(optimizer, step_size=6, gamma=0.6) #decay learning rate by 0.7 every 8 epochs

In [16]:
# Model settings
num_epochs = 100
train_losses = []
val_losses = []

# Early stopping
best_val_loss = float('inf')
patience = 7
counter = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Move data to the appropriate device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        train_outputs = model(X_batch)
        # loss = loss_function(train_outputs, y_batch)
        loss = weighted_loss_function(y_batch, train_outputs)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model parameters
        epoch_loss += loss.item()  # Accumulate epoch loss

    # Step the scheduler
    scheduler.step()  # Adjust learning rate

    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            # Move data to the appropriate device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            val_outputs = model(X_batch)
            # val_loss += loss_function(val_outputs, y_batch).item()
            val_loss += weighted_loss_function(y_batch, val_outputs).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{feature}_{hemi}_id{id}.pt')
    else:
        counter += 1
        if counter == patience:
            print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
            break

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss (MAWE): {train_loss:.4f}, Val Loss (MAWE): {val_loss:.4f}, Best Val Loss: {best_val_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.1e}')


Epoch [1/100], Train Loss (MAWE): 1.3122, Val Loss (MAWE): 1.4160, Best Val Loss: 1.4160, LR: 1.0e-03
Epoch [2/100], Train Loss (MAWE): 1.2325, Val Loss (MAWE): 1.3091, Best Val Loss: 1.3091, LR: 1.0e-03
Epoch [3/100], Train Loss (MAWE): 1.1510, Val Loss (MAWE): 1.2122, Best Val Loss: 1.2122, LR: 1.0e-03
Epoch [4/100], Train Loss (MAWE): 1.0973, Val Loss (MAWE): 1.1824, Best Val Loss: 1.1824, LR: 1.0e-03
Epoch [5/100], Train Loss (MAWE): 1.0812, Val Loss (MAWE): 1.1688, Best Val Loss: 1.1688, LR: 1.0e-03
Epoch [6/100], Train Loss (MAWE): 1.0714, Val Loss (MAWE): 1.1638, Best Val Loss: 1.1638, LR: 6.0e-04
Epoch [7/100], Train Loss (MAWE): 1.0503, Val Loss (MAWE): 1.1278, Best Val Loss: 1.1278, LR: 6.0e-04
Epoch [8/100], Train Loss (MAWE): 1.0427, Val Loss (MAWE): 1.1211, Best Val Loss: 1.1211, LR: 6.0e-04
Epoch [9/100], Train Loss (MAWE): 1.0373, Val Loss (MAWE): 1.1172, Best Val Loss: 1.1172, LR: 6.0e-04
Epoch [10/100], Train Loss (MAWE): 1.0323, Val Loss (MAWE): 1.1161, Best Val Loss:

In [17]:
test_model = LSTM(lookback, hs, sl)
test_model.load_state_dict(torch.load(f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{feature}_{hemi}_id{id}.pt'))
test_model.to('cpu')
test_model.eval()

# List to store predictions
predictions = []

# Perform predictions in batches
with torch.no_grad():
    batch_size = 128
    for i in range(0, len(X_test), batch_size):
        X_batch = X_test[i:i+batch_size]
        
        # Forward pass
        y_pred = test_model(X_batch)
        
        # Append the predictions (already on CPU)
        predictions.append(y_pred.numpy())

# Concatenate all predictions into a single array
predictions = np.concatenate(predictions)

In [18]:
def batch_mae(y_test, predictions, batch_size):
    mae_accum = []
    
    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        mae_accum.append(np.mean(np.abs(y_batch - pred_batch)))

    return np.mean(mae_accum)

def batch_nmae_std(y_test, predictions, batch_size):
    overall_std = np.std(y_test)  # Use the overall standard deviation
    nmae_accum = []

    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        nmae_accum.append(np.mean(np.abs(y_batch - pred_batch)) / overall_std)

    return np.mean(nmae_accum)

def batch_corrcoef(y_test, predictions, batch_size):
    corr_accum = []
    
    for i in range(0, len(y_test), batch_size):
        y_batch = y_test[i:i+batch_size]
        pred_batch = predictions[i:i+batch_size]
        if len(y_batch) > 1:  # Corrcoef needs at least 2 points to calculate
            corr_accum.append(np.corrcoef(y_batch.flatten(), pred_batch.flatten())[0, 1])
    
    return np.mean(corr_accum)

def skill(m, o):
    skill = 1 - (np.sum((m - o)**2) / np.sum((o - np.mean(o))**2))
    return skill      

# Set the batch size
batch_size = 10000

# Calculate metrics in batches
MAE = batch_mae(y_test, predictions, batch_size)
NMAE_std = batch_nmae_std(y_test, predictions, batch_size)  # Now using overall std of y_test
R = batch_corrcoef(y_test, predictions, batch_size)

# Calculate skill normally (if this isn't memory intensive)
PE = skill(predictions, y_test)

print(f'Test Loss (MAE): {MAE:.3f}, Test Loss (NMAE_std): {NMAE_std:.3f}, R: {R:.2f}, PE: {PE:.4f}')
#print(f'Test Loss (MAE): {MAE:.4f}, Test Loss (NMAE_std): {NMAE_std:.4f}')


Test Loss (MAE): 0.035, Test Loss (NMAE_std): 0.346, R: 0.73, PE: 0.5281
